## Physical Geography

In [12]:
%load_ext autoreload
%autoreload 2

from world_simulator import run_river_skeleton_pipeline, test_noise, generate_fractal_noise
from world_simulator import GPUThermalEroder, CoastalTaper

from world_simulator import HydrologyAnalyzer, validate_arrow_directions, plot_flow, CalculateFlowMagnitude, plot_river_hierarchy, assign_river_widths, plot_river_physics, save_hydro_network, validate_topology_continuity




import os
import geopandas as gpd
import numpy as np
import rasterio
from gdgtm import change_raster_res


vector_src_dir = "/home/pete/Documents/wfrp/source_vectors/"
raster_src_dir = "/home/pete/Documents/wfrp/source_rasters/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## River and sea setup

Here we take a georeferenced, trimmed corase altitude map, we load it, increase resolution, and then we smooth it.

We use an erosion approach to overal smoothing and aim for Int16 data type - which gives us some serious mileage in the "disk saving department" - 100s of MBs.

The exact maths are indicated in the class docstring for the GPUThermalEroder. For now it is relevant to note that we went for a short erosion (100 steps), slow erosion rate (0.05), and a high threshold of 2. What this means is that we preserve most of the high ground on the map, while introducing just a little smoothing around the slopes to remove the blockiness. Not perfect, but the step below will take care of doing perfect :)



### Vectorizing the river net from QGIS polys

The goal here is to take a bunch of QGIS polygons and change them into an actual net of lines for downstream processing.

In [ ]:
# if __name__ == "__main__":
#     # Example configuration
#     IN_FILE = os.path.join(vector_src_dir, "wfrp_empire_rivers_poly.gpkg")
#     OUT_FILE = os.path.join(vector_src_dir, "wfrp_empire_rivers_line.gpkg")
    
#     # Config
#     GAP_TOLERANCE = 3500     # 20km gap filling
#     INTERP_DISTANCE = 250     # 500m vertex resolution
#     PRUNE = 3500
    
#     if os.path.exists(IN_FILE):
#         run_river_skeleton_pipeline(IN_FILE, OUT_FILE, GAP_TOLERANCE, INTERP_DISTANCE, PRUNE)
#     else:
#         print(f"File not found: {IN_FILE}")

In [ ]:
# ### Phase 1: Make the rivers flow and make them have the right size.

# if __name__ == "__main__":

#     # rivers_path = os.path.join(vector_src_dir, "wfrp_empire_rivers_line.gpkg")
#     sea_path = os.path.join(vector_src_dir, "wfrp_empire_sea_poly.gpkg")
#     lakes_path = os.path.join(vector_src_dir, "wfrp_empire_lakes_poly.gpkg")
#     rivers_path = os.path.join(vector_src_dir, "wfrp_empire_rivers_line.gpkg")
    
#     rivers = gpd.read_file(rivers_path) # The output from skeletonization
#     sea = gpd.read_file(sea_path)       # You need this
#     lakes = gpd.read_file(lakes_path)   # You need this (Canonical lakes)

#     ### Step 1: Orient the rivers in the correct direction.
#     analyzer = HydrologyAnalyzer(rivers, sea, lakes)
#     oriented_rivers = analyzer.run()
#     validate_arrow_directions(oriented_rivers, analyzer.G)
#     # plot_flow(oriented_rivers, sea, lakes)

#     ### Step 2: Establish river hierarchy
#     # 1. Get the Directed Graph from the Phase 1 Analyzer
#     flow_calculator = CalculateFlowMagnitude(analyzer.DiG)
#     # 2. Calculate Magnitude
#     dag_with_flow = flow_calculator.run()
#     # 3. Visualize
#     # plot_river_hierarchy(dag_with_flow, sea, lakes)

In [ ]:
# if __name__ == "__main__":
#     OUT_FILE = os.path.join(vector_src_dir, "wfrp_empire_major_rivers_net.gpkg")
#     ### Step 3: Work out river widths:
#     rivers_with_width = assign_river_widths(
#         dag_with_flow, 
#         min_width=20.0,    # Source streams are 20m wide
#         max_width=1000.0,   # The Reik is 800m wide at Altdorf - we make it 1000 for our 250m grid.
#         scale_factor=100.0  # Multiplier for the log growth
#     )
    
#     # 2. Restore CRS (Important for buffering!)
#     rivers_with_width.set_crs(rivers.crs, inplace=True)
    
#     # 3. Visualize
#     # plot_river_physics(rivers_with_width, sea, lakes)
#     save_hydro_network(rivers_with_width, OUT_FILE)

In [4]:
# validate_topology_continuity(rivers_with_width)

--- TOPOLOGY CONTINUITY CHECK ---
Checked 100 junctions.
SUCCESS: Water flows continuously from line to line.


### Sea set-up

Here we prep the sea



In [1]:
# =============================================================================
#  IMPORTS
# =============================================================================

%load_ext autoreload
%autoreload 2

import os
import numpy as np
import geopandas as gpd
import rasterio

# from world_simulator.terrain_engine import CoordinateEngine
# Added necessary utility functions for Step 2
from world_simulator.misc_utils import (
    vector_to_mask, 
    generate_fractal_mask, 
    measure_fractal_dimension,
    repair_mask_artifacts
)


/home/pete/miniconda/envs/gdgtm_dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251224_074409.log
INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251224_074410.log
INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251224_074410.log
INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251224_074410.log


In [2]:
# =============================================================================
#  STEP 1: LOAD CONTEXT DATA
# =============================================================================

vector_src = "/home/pete/Documents/wfrp/source_vectors"
raster_src = "/home/pete/Documents/wfrp/source_rasters"

# A. The Base DEM (Canvas & Starting Heights)
base_dem_path = os.path.join(raster_src, "wfrp_empire_smoothed_topo.tif") # From your previous step
with rasterio.open(base_dem_path) as src:
    base_profile = src.profile
    base_shape = (src.height, src.width)
    base_elevation = src.read(1) # Start with existing terrain

# B. Vectors
rivers_gdf = gpd.read_file(os.path.join(vector_src, "wfrp_empire_major_rivers_net.gpkg"))
lakes_gdf = gpd.read_file(
    os.path.join(vector_src, "wfrp_empire_lakes_poly.gpkg"),
    layer="wfrp_empire_lakes")
sea_gdf = gpd.read_file(os.path.join(vector_src, "wfrp_empire_sea_poly.gpkg")) # Explicit Sea Load



In [3]:
# =============================================================================
#  STEP 2: PRE-PROCESS COAST (Measure & Mimic)
# =============================================================================
# --- A. COASTAL ANALYSIS ---
print("Analyzing Coastline Physics...")

# 1. Rasterize (Get the shape)
raw_sea_mask = vector_to_mask(sea_gdf, base_profile)

# Define the artifact zones (Map edges/NoData boundaries)
# Format: (West_Col, North_Row, East_Col, South_Row)
artifact_boxes = [
    (0, 1050, 100, 1300),   # Box 1: West edge artifact
    (3500, 0, 7100, 900)    # Box 2: Northern edge artifact
]

# 2. Measure the "Native" Fractal Dimension
# We measure the complexity at coarse scales (e.g., 2km to 64km).
# This tells us if the coast is "Scottish" (High D) or "Floridian" (Low D).
# We stop at min_scale=16px (~2km) to avoid reading the square pixel artifacts.
measured_d = measure_fractal_dimension(
    mask=raw_sea_mask,
    min_scale=16,   # Ignore details smaller than ~2km
    max_scale=512,  # Measure up to ~64km bays
    exclusion_boxes=artifact_boxes  # <--- Plugged in here
)

# print(f"Measured Fractal Dimension: {measured_d}")

# Safety Clamp: Real coastlines rarely exceed 1.5 or drop below 1.1
measured_d = np.clip(measured_d, 1.1, 1.45)
print(f"  > Detected Fractal Dimension: {measured_d:.3f}")

# 3. Synthesize Detail
# We use the measured D to generate the sub-pixel details.
fractal_sea_mask = generate_fractal_mask(
    mask=raw_sea_mask,
    fractal_dimension=measured_d,  # <--- The Measurement informs the Synthesis
    scale=20.0,
    seed=123
)

# 4. Repair Boundaries
# We surgically restore the straight edges in the artifact boxes
# to prevent the fractal noise from 'chewing' up the map border.
print("Repairing map boundary artifacts...")
fractal_sea_mask = repair_mask_artifacts(
    fractal_mask=fractal_sea_mask,
    reference_mask=raw_sea_mask,      # The clean original
    boxes=artifact_boxes,             # <--- Plugged in here again
    base_dem=base_elevation,          # Check for valid data
    nodata_value=base_profile.get('nodata', -32768)
)

Analyzing Coastline Physics...
INFO: Rasterizing 11 vector features into mask...
  > Detected Fractal Dimension: 1.224
Repairing map boundary artifacts...
INFO: Repairing artifacts in 2 zones...


In [4]:
# =============================================================================
#  DEBUG EXPORT: FRACTAL SEA MASK
# =============================================================================
output_mask_path = os.path.join(raster_src, "wfrp_coast_mask_test.tif")
print(f"Exporting debug mask to {output_mask_path}...")

# Prepare Profile
mask_profile = base_profile.copy()
mask_profile.update({
    'dtype': 'uint8',
    'count': 1,
    'compress': 'lzw',
    'nodata': 0 
})

with rasterio.open(output_mask_path, 'w', **mask_profile) as dst:
    # Cast Boolean (True/False) to Uint8 (1/0)
    # OPTIONAL: Multiply by 255 so it appears as Black/White in QGIS immediately
    # without needing to stretch the histogram manually.
    debug_data = fractal_sea_mask.astype('uint8') * 255
    
    dst.write(debug_data, 1)

print("Debug export complete.")

Exporting debug mask to /home/pete/Documents/wfrp/source_rasters/wfrp_coast_mask_test.tif...
Debug export complete.


## Topography: 

The plan is outlined here: https://github.com/pete-jacobsson/enemy_rebuilt/wiki/Topography

### PHASE 1: IMPORTS & RAW DATA INGESTION

In [1]:
%load_ext autoreload
%autoreload 2


import gc
import geopandas as gpd

import numpy as np
import os

import pandas as pd
import rasterio
from rasterio.plot import show

from world_simulator import (
    rasterize_vector_to_disk,
    rasterize_variable_width_rivers,
    rasterize_uint8_to_disk
)

/home/pete/miniconda/envs/gdgtm_dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251225_164957.log
INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251225_164958.log
INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251225_164959.log
INFO: Logging initialized. Writing to: logs/wfrp_phys_geo_20251225_164959.log


In [2]:
# 1. Configuration: File Paths
# -----------------------------------------------------------------------------
INPUT_DIR = "/home/pete/Documents/wfrp"

# File definitions matching your manifest
files = {
    "gcm":    os.path.join(INPUT_DIR, "source_vectors/wfrp_empire_geo_control.gpkg"),  # 5-Zone Vector Mask
    "dem":    os.path.join(INPUT_DIR, "source_rasters/wfrp_empire_topo_high_res.tif"),  # Coarse Heightmap
    "rivers": os.path.join(INPUT_DIR, "source_vectors/wfrp_empire_major_rivers_net.gpkg"),        # River Vectors
    "lakes":  os.path.join(INPUT_DIR, "source_vectors/wfrp_empire_lakes_high_res_poly.gpkg"),         # Lake Polygons
    "sea":    os.path.join(INPUT_DIR, "source_vectors/wfrp_empire_sea_high_res_poly.gpkg")            # Sea Polygons
}

# 2. Load Raster Data (The Base DEM)
# -----------------------------------------------------------------------------
# We keep the source open or read immediately into memory. 
# Here we read the data and profile to establish the baseline grid/CRS.
try:
    with rasterio.open(files['dem']) as src:
        base_profile = src.profile   # Metadata (CRS, Transform, Width, Height)
        base_bounds = src.bounds     # Physical extent
    
    print(f"SUCCESS: Loaded Base DEM.")
    print(f"  - Shape: {base_dem_data.shape}")
    print(f"  - CRS: {base_profile['crs']}")
    print(f"  - Bounds: {base_bounds}")

except Exception as e:
    print(f"ERROR: Could not load DEM. {e}")

# 3. Load Vector Data (Geological Mask, Hydro, Coastline)
# -----------------------------------------------------------------------------
# We load these as GeoDataFrames. No rasterization yet.
try:
    # A. Geological Control Map (Zone 1-5)
    gdf_gcm = gpd.read_file(files['gcm'])
    print(f"SUCCESS: Loaded GCM ({len(gdf_gcm)} polygons).")

    # B. Hydrology
    gdf_rivers = gpd.read_file(files['rivers'])
    print(f"SUCCESS: Loaded Rivers ({len(gdf_rivers)} segments).")

    gdf_lakes = gpd.read_file(files['lakes'])
    print(f"SUCCESS: Loaded Lakes ({len(gdf_lakes)} polygons).")

    # C. Coastline / Sea
    gdf_sea = gpd.read_file(files['sea'])
    print(f"SUCCESS: Loaded Sea Mask ({len(gdf_sea)} polygons).")

except Exception as e:
    print(f"ERROR: Could not load vector data. {e}")

# 4. Quick CRS Consistency Check
# -----------------------------------------------------------------------------
# All vectors must match the DEM's CRS.
vectors = {'gcm': gdf_gcm, 'rivers': gdf_rivers, 'lakes': gdf_lakes, 'sea': gdf_sea}
dem_crs = base_profile['crs']

print("\n--- CRS Consistency Check ---")
for name, gdf in vectors.items():
    if gdf.crs != dem_crs:
        print(f"WARNING: {name} CRS ({gdf.crs}) does not match DEM ({dem_crs}). Reprojection required later.")
    else:
        print(f"OK: {name} matches DEM CRS.")

SUCCESS: Loaded Base DEM.
ERROR: Could not load DEM. name 'base_dem_data' is not defined
SUCCESS: Loaded GCM (15 polygons).
SUCCESS: Loaded Rivers (112264 segments).
SUCCESS: Loaded Lakes (80 polygons).
SUCCESS: Loaded Sea Mask (493 polygons).

--- CRS Consistency Check ---
OK: gcm matches DEM CRS.
OK: rivers matches DEM CRS.
OK: lakes matches DEM CRS.
OK: sea matches DEM CRS.


In [3]:
# =============================================================================
#  PHASE 1.1: PREP SIMPLE MASKS AND SAVE TO DISK
# =============================================================================

# Configuration
OUTPUT_MASK_DIR = os.path.join(INPUT_DIR, "processed_masks")
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)

# Define processing targets
# Format: (Input Vector Key, Output Filename, All Touched?)
# Note: 'All Touched=True' is usually better for Lakes/Sea to ensure 
# thin coastal features don't disappear.
targets = [
    ("sea", "mask_sea.tif", True),
    ("lakes", "mask_lakes.tif", True)
]

print("Starting Rasterization Sequence...")

for key, filename, touch_setting in targets:
    output_path = os.path.join(OUTPUT_MASK_DIR, filename)
    
    # Check if we already loaded it in previous cells, otherwise use path
    source = vectors[key] if 'vectors' in locals() and vectors.get(key) is not None else files[key]
    
    # Execute Function
    rasterize_vector_to_disk(
        vector_source=source,
        template_raster_path=files['dem'],
        output_path=output_path,
        burn_value=1,
        all_touched=touch_setting
    )



Starting Rasterization Sequence...
Rasterizing to mask_sea.tif...
Saved: /home/pete/Documents/wfrp/processed_masks/mask_sea.tif
Rasterizing to mask_lakes.tif...
Saved: /home/pete/Documents/wfrp/processed_masks/mask_lakes.tif


In [4]:
# =============================================================================
#  PHASE 1.2: RASTERIZE VARIABLE WIDTH RIVERS
# =============================================================================

# Configuration
OUTPUT_MASK_DIR = os.path.join(INPUT_DIR, "processed_masks")
RIVER_OUTPUT_FILE = "mask_rivers_variable.tif"
OUTPUT_PATH = os.path.join(OUTPUT_MASK_DIR, RIVER_OUTPUT_FILE)

print("Starting River Rasterization...")

# Check if 'vectors' dict exists from previous cells, otherwise load from file
source = vectors['rivers'] if 'vectors' in locals() and vectors.get('rivers') is not None else files['rivers']

# Execute the variable width rasterization
# We use the 'width' column identified in your printout
rasterize_variable_width_rivers(
    vector_source=source,
    template_raster_path=files['dem'],
    output_path=OUTPUT_PATH,
    width_col='width'
)


Starting River Rasterization...
Buffering river segments by width...
Rasterizing variable widths to mask_rivers_variable.tif...
Saved: /home/pete/Documents/wfrp/processed_masks/mask_rivers_variable.tif


In [5]:
# =============================================================================
#  PHASE 1.3: RASTERIZE GEOLOGICAL CONTROL MAP (GCM)
# =============================================================================
OUTPUT_MASK_DIR = os.path.join(INPUT_DIR, "processed_masks")
GCM_OUTPUT_FILE = "mask_geology.tif"
OUTPUT_PATH = os.path.join(OUTPUT_MASK_DIR, GCM_OUTPUT_FILE)

print("Starting GCM Rasterization...")

# Source Retrieval
source = vectors['gcm'] if 'vectors' in locals() and vectors.get('gcm') is not None else files['gcm']

# Execute Rasterization
# Using 'DN' as identified in your printout
rasterize_uint8_to_disk(
    vector_source=source,
    template_raster_path=files['dem'],
    output_path=OUTPUT_PATH,
    value_col='DN'
)

Starting GCM Rasterization...
Rasterizing Geological Zones (Col: DN) to mask_geology.tif...
Saved: /home/pete/Documents/wfrp/processed_masks/mask_geology.tif


In [6]:
# =============================================================================
#  PHASE 1 CLEANUP
# =============================================================================
print("\nFinalizing Phase 1 Memory Cleanup...")

if 'vectors' in locals():
    del vectors

if 'base_profile' in locals():
    del base_profile

if 'base_bounds' in locals():
    del base_bounds


# Force GC
gc.collect()

print("Phase 1 Complete. All vectors rasterized to 'processed_masks/' and memory cleared.")



Finalizing Phase 1 Memory Cleanup...
Phase 1 Complete. All vectors rasterized to 'processed_masks/' and memory cleared.


### PHASE 2: Macro-Sculpting (The Foundation)

In [18]:
%load_ext autoreload
%autoreload 2

import rasterio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# 1. Configuration: File Paths
# -----------------------------------------------------------------------------
INPUT_DIR = "/home/pete/Documents/wfrp"

# File definitions matching your manifest
files = {
    "gcm":    os.path.join(INPUT_DIR, "processed_masks/mask_geology.tif"),  # 5-Zone Vector Mask
    "dem":    os.path.join(INPUT_DIR, "source_rasters/wfrp_empire_topo_high_res.tif"),  # Coarse Heightmap
}

# 2. Load Raster Data (The Base DEM)
# -----------------------------------------------------------------------------
# We keep the source open or read immediately into memory. 
# Here we read the data and profile to establish the baseline grid/CRS.
try:
    with rasterio.open(files['dem']) as src:
        dem = src.read(1)
        base_profile = src.profile   # Metadata (CRS, Transform, Width, Height)
        base_bounds = src.bounds     # Physical extent

    with rasterio.open(files['gcm']) as src:
        gcm = src.read(1)

    print("dem and gcm loaded!")

except Exception as e:
    print(f"ERROR: Could not load DEM. {e}")

dem and gcm loaded!


#### STEP A: De-stepping
The Canonical DEM is quantified into 5 levels. We apply a wide-radius Gaussian Blur to the base elevation to turn these "steps" into continuous slopes, creating a smooth canvas for detailing.

In [20]:
%load_ext autoreload
%autoreload 2

import os
import rasterio
import numpy as np
from world_simulator import destep_terrain

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# 1. Configuration
# -----------------------------------------------------------------------------
# Adjust radius based on your resolution. 
# If 1px ~= 125m, a 64px radius creates ~6km wide slopes.
# Larger radius = smoother, wider ramps between the steps.
BLUR_RADIUS = 64 
ITERATIONS = 3
SCALE_FACTOR = 2  # 1m = 2 units (so 50m = 100 units)

OUTPUT_DIR = os.path.join(INPUT_DIR, "source_rasters")
OUTPUT_FILENAME = "phase2_a_destep.tif"
OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [28]:
# 2. Execution
if 'dem' not in locals():
    raise ValueError("Variable 'dem' not found. Load Phase 1 data first.")

# Extract Nodata value from the loaded profile
# If not defined in file, default to -32768 (standard for int16)
src_nodata = base_profile.get('nodata', None)
if src_nodata is None:
    src_nodata = -32768
    print(f"Warning: No Nodata in profile. Assuming {src_nodata}.")
else:
    print(f"Using Nodata Value: {src_nodata}")

# Run Nodata-Aware Smoothing
smoothed_dem = destep_terrain(
    dem_array=dem,
    nodata_value=src_nodata,
    radius=BLUR_RADIUS, 
    iterations=ITERATIONS, 
    scale_factor=SCALE_FACTOR
)

Using Nodata Value: -32768.0
--- Starting Nodata-Aware De-stepping (R=64, I=3) ---
  > Blur Pass 1/3...
  > Blur Pass 2/3...
  > Blur Pass 3/3...
Restoring Nodata and quantizing...


In [29]:
# 3. Save to Compressed GeoTIFF
# -----------------------------------------------------------------------------
print(f"Saving result to {OUTPUT_FILENAME}...")

# Update metadata for the new format
profile = base_profile.copy()
profile.update({
    'driver': 'GTiff',
    'dtype': rasterio.int16,
    'count': 1,
    'compress': 'lzw',
    'predictor': 2,  # Horizontal differencing (good for elevation data)
    'nodata': -32768 # Standard int16 nodata value
})

with rasterio.open(OUTPUT_PATH, 'w', **profile) as dst:
    dst.write(smoothed_dem, 1)

print(f"SUCCESS: De-stepping complete.")
print(f"  - Output shape: {smoothed_dem.shape}")
print(f"  - Min/Max values: {np.min(smoothed_dem)} / {np.max(smoothed_dem)}")
print(f"  - Scale: 1 unit = {1/SCALE_FACTOR} meters")

Saving result to phase2_a_destep.tif...
SUCCESS: De-stepping complete.
  - Output shape: (10562, 16395)
  - Min/Max values: -32768 / 32424
  - Scale: 1 unit = 0.5 meters


In [30]:
# 4. Cleanup Memory
# -----------------------------------------------------------------------------
# We delete the smoothed array from RAM. 
# We keep 'dem' (original) and 'gcm' for the next prompt (Step B).
del dem
import gc
gc.collect()

1044

In [16]:
# =============================================================================
#  DEBUG: VISUALIZE THE WARP FIELD (Graph Paper Mode)
# =============================================================================
print("Generating Warp Debug Grid...")

# 1. Access the internal coordinate fabric
# These should have been modified by the RiverWarpLayer
warped_x = engine.coords_x
warped_y = engine.coords_y

# 2. Generate a Grid Pattern
# We create white lines every 500 pixels
spacing = 500      # Size of the grid squares (pixels)
thickness = 40     # Thickness of the lines (pixels)

# Logic: If the coordinate modulo spacing is less than thickness, draw a line.
# This draws lines based on WHERE the pixel "thinks" it is in the warped space.
vert_lines = (warped_x % spacing) < thickness
horiz_lines = (warped_y % spacing) < thickness

# Combine Vertical + Horizontal
grid_pattern = np.logical_or(vert_lines, horiz_lines)

# 3. Export
debug_path = "/home/pete/Documents/wfrp/debug_folding.tif"
with rasterio.open(debug_path, 'w', **base_profile) as dst:
    # Convert Boolean (True/False) to Uint8 (255/0) for visibility
    debug_img = grid_pattern.astype(np.uint8) * 255
    dst.write(debug_img, 1)

print(f"Check {debug_path} in QGIS.")
print("If the grid lines are perfectly straight, the warp is BROKEN.")
print("If the grid lines curve near rivers, the warp WORKS.")

Generating Warp Debug Grid...
Check /home/pete/Documents/wfrp/debug_folding.tif in QGIS.
If the grid lines are perfectly straight, the warp is BROKEN.
If the grid lines curve near rivers, the warp WORKS.


What this will not do is getting a good coastal taper. For this we use the CoastalTaper class.
For this we set ourselves at 250 pixels from the shore (which means base DEM elevation is reached some 27.25km from the Sea of Claws). The power is the default of 2 - this means a wide coastal plain and a steep rise that still preserves the hills in the north of Nordland.